In [2]:
!pip install datasets
#clean output
from IPython.display import clear_output
clear_output()

In [3]:
import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0,2,3,4,5,6,7'

In [4]:
from datasets import load_dataset

dataset = load_dataset("gyr66/privacy_detection")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = dataset["train"].train_test_split(train_size=0.8, seed=42)
dataset["validation"] = dataset.pop("test")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2012
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 503
    })
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "hfl/chinese-roberta-wwm-ext-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    return [-100 if word_id is None else labels[word_id] for word_id in word_ids]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=512
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        print("all_labels",all_labels)
        print("i, labels", i, " ",labels)
        print("len(all_labels)",len(all_labels))
        word_ids = tokenized_inputs.word_ids(i)
        print("word_ids",word_ids)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    assert all([len(labels) == len(input_ids) for labels, input_ids in zip(tokenized_inputs["labels"], tokenized_inputs["input_ids"])])
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
    num_proc=16
)

Map (num_proc=16):   0%|          | 0/2012 [00:00<?, ? examples/s]

all_labels all_labels[[1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 0], [3, 4, 4, 4, 0, 0, 0, 0, 13, 14, 14, 14, 14, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
ner_feature = dataset["train"].features["ner_tags"]
label_names = ner_feature.feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
ner_feature

In [ ]:
label_names

In [ ]:
id2label

In [ ]:
label2id

In [ ]:
# import numpy as np


# def compute_metrics(eval_preds):
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
#     return {
#         "precision": all_metrics["overall_precision"],
#         "recall": all_metrics["overall_recall"],
#         "f1": all_metrics["overall_f1"],
#         "accuracy": all_metrics["overall_accuracy"],
#     }

In [ ]:
# from transformers import AutoModelForTokenClassification
# from model import BERT_CRF_ForTokenClassification

# model = BERT_CRF_ForTokenClassification.from_pretrained(
#     model_checkpoint,
#     id2label=id2label,
#     label2id=label2id,
# )

In [ ]:
# from peft import  get_peft_model, LoraConfig, TaskType


# peft_config = LoraConfig(
#     task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1, bias="none"
# )

In [ ]:
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [ ]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     "chinese-roberta-wwm-ext-large-lora-crf-ner",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=1e-3,
#     num_train_epochs=100,
#     weight_decay=0.01,
#     push_to_hub=True,
#     per_device_train_batch_size=4,
#     logging_strategy="epoch",
#     dataloader_num_workers=16,
#     metric_for_best_model="f1",
#     load_best_model_at_end=True,
#     save_total_limit=1
# )

In [ ]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["validation"],
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
# )
# trainer.train()

In [ ]:
# trainer.push_to_hub(commit_message="Training 10 Epochs and save the best F1 model", language="zh", dataset_tags="gyr66/privacy_detection")